In [18]:
import os
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
particles_df = pd.read_csv('datasets/unarch/train_100_events/event000001000-particles.csv')
hits_df = pd.read_csv('datasets/unarch/train_100_events/event000001000-hits.csv')
cells_df = pd.read_csv('datasets/unarch/train_100_events/event000001000-cells.csv')
truth_df = pd.read_csv('datasets/unarch/train_100_events/event000001000-truth.csv')

In [3]:
print(particles_df.shape)
particles_df.head(4)

(12263, 9)


,particle_id,vx,vy,vz,px,py,pz,q,nhits
0,4503668346847232,-0.009288,0.009861,-0.077879,-0.055269,0.323272,-0.203492,-1,8
1,4503737066323968,-0.009288,0.009861,-0.077879,-0.948125,0.470892,2.010060,1,11
2,4503805785800704,-0.009288,0.009861,-0.077879,-0.886484,0.105749,0.683881,-1,0
3,4503874505277440,-0.009288,0.009861,-0.077879,0.257539,-0.676718,0.991616,1,12


In [4]:
print(hits_df.shape)
hits_df.head(4)

(120939, 7)


,hit_id,x,y,z,volume_id,layer_id,module_id
0,1,-64.4099,-7.163700,-1502.5,7,2,1
1,2,-55.3361,0.635342,-1502.5,7,2,1
2,3,-83.8305,-1.143010,-1502.5,7,2,1
3,4,-96.1091,-8.241030,-1502.5,7,2,1


In [5]:
print(cells_df.shape)
cells_df.head(4)

(664996, 4)


,hit_id,ch0,ch1,value
0,1,209,617,0.013832
1,1,210,617,0.079887
2,1,209,618,0.211723
3,2,68,446,0.334087


In [6]:
print(truth_df.shape)
truth_df.head(4)

(120939, 9)


,hit_id,particle_id,tx,ty,tz,tpx,tpy,tpz,weight
0,1,0,-64.4116,-7.164120,-1502.5,250710.000000,-149908.000000,-956385.00000,0.000000
1,2,22525763437723648,-55.3385,0.630805,-1502.5,-0.570605,0.028390,-15.49220,0.000010
2,3,0,-83.8280,-1.145580,-1502.5,626295.000000,-169767.000000,-760877.00000,0.000000
3,4,297237712845406208,-96.1229,-8.230360,-1502.5,-0.225235,-0.050968,-3.70232,0.000008


In [7]:
detectors_df = pd.read_csv('datasets/unarch/detectors.csv')

In [8]:
print(detectors_df.shape)
detectors_df.head(4)

(18728, 21)


,volume_id,layer_id,module_id,cx,cy,cz,rot_xu,rot_xv,rot_xw,rot_yu,...,rot_yw,rot_zu,rot_zv,rot_zw,module_t,module_minhu,module_maxhu,module_hv,pitch_u,pitch_v
0,7,2,1,-65.7965,-5.17830,-1502.5,0.078459,-0.996917,0.0,-0.996917,...,0.0,0,0,-1,0.15,8.4,8.4,36,0.05,0.05625
1,7,2,2,-139.8510,-6.46568,-1502.0,0.046183,-0.998933,0.0,-0.998933,...,0.0,0,0,-1,0.15,8.4,8.4,36,0.05,0.05625
2,7,2,3,-138.6570,-19.34190,-1498.0,0.138156,-0.990410,0.0,-0.990410,...,0.0,0,0,-1,0.15,8.4,8.4,36,0.05,0.05625
3,7,2,4,-64.1764,-15.40740,-1498.0,0.233445,-0.972370,0.0,-0.972370,...,0.0,0,0,-1,0.15,8.4,8.4,36,0.05,0.05625


In [9]:
submission_df = pd.read_csv('datasets/unarch/sample_submission.csv')

In [10]:
print(submission_df.shape)
submission_df.head(4)

(13741466, 3)


,event_id,hit_id,track_id
0,0,1,0
1,0,2,0
2,0,3,0
3,0,4,0


In [11]:
train_dataset_dir = 'datasets/unarch/train_100_events'
dataset_filenames = os.listdir(train_dataset_dir)
print(dataset_filenames[:4])
event_ids = []
event_filenames = {}
for filename in dataset_filenames[:20]:
    event_id = filename[5:14]
    print(event_id)
    if event_id not in event_ids:
        event_ids.append(event_id)
    if event_id not in event_filenames:
        event_filenames[event_id] = [filename]
    else:
        event_filenames[event_id].append(filename)

['event000001015-truth.csv', 'event000001022-cells.csv', 'event000001056-cells.csv', 'event000001043-particles.csv']
000001015
000001022
000001056
000001043
000001061
000001035
000001026
000001004
000001081
000001076
000001008
000001057
000001066
000001019
000001083
000001053
000001090
000001001
000001074
000001085


In [19]:
def random_sublist_select(original_list, sublist_size):
    return [original_list[i] for i in random.sample(range(len(original_list)), sublist_size)]

In [20]:
def offset_sublist_select(original_list, sublist_size, offset):
    return original_list[offset:offset + sublist_size]

In [21]:
def select_random_indexses_subset(size, subset_size):
    return random.sample(tuple(range(size)), subset_size) 

In [22]:
def select_offset_indexses_subset(size, subset_size, offset):
    return tuple(range(size))[offset:offset + subset_size]

In [25]:
def read_dataset_filenames_from_dir(path_to_datasets_dir):
    dataset_filenames = os.listdir(path_to_datasets_dir)
    event_filenames = {}
    for filename in dataset_filenames:
        event_id = filename[5:14]
        if event_id not in event_filenames:
            event_filenames[event_id] = [filename]
        else:
            event_filenames[event_id].append(filename)
    return event_filenames

In [30]:
def select_events(indexes_list, event_names):
    return tuple(event_names[i] for i in indexes_list)

In [31]:
def random_events_select(event_names, subset_size):
    event_names_len = len(event_names)
    indexes = select_random_indexses_subset(event_names_len, subset_size)
    return select_events(indexes, event_names)

In [32]:
def offset_events_select(event_names, subset_size, offset):
    event_names_len = len(event_names)
    indexes = select_offset_indexses_subset(event_names_len, subset_size, offset)
    return select_events(indexes, event_names)

In [36]:
def read_dataset_filenames_random(directory_list, sample_size=0):
    event_grouped_dataset_filenames = {}
    for directory in directory_list:
        dataset_filenames = read_dataset_filenames_from_dir(directory)
        if sample_size > 0:
            event_names = random_events_select(tuple(dataset_filenames), sample_size)
            dataset_filenames = {event_name: dataset_filenames[event_name] for event_name in event_names}
        event_grouped_dataset_filenames.update(dataset_filenames)
    return event_grouped_dataset_filenames    

In [37]:
def read_dataset_filenames_offset(directory_list, sample_size=0, offset=0):
    event_grouped_dataset_filenames = {}
    for directory in directory_list:
        dataset_filenames = read_dataset_filenames_from_dir(directory)
        if sample_size > 0:
            event_names = offset_events_select(tuple(dataset_filenames), sample_size, offset)
            dataset_filenames = {event_name: dataset_filenames[event_name] for event_name in event_names}
        event_grouped_dataset_filenames.update(dataset_filenames)
    return event_grouped_dataset_filenames  

In [38]:
event_grouped_dataset_filenames = read_dataset_filenames_random(['datasets/unarch/train_1/'], 20)

In [39]:
event_grouped_dataset_filenames

{'000001018': ['event000001018-hits.csv',
  'event000001018-cells.csv',
  'event000001018-truth.csv',
  'event000001018-particles.csv'],
 '000001052': ['event000001052-truth.csv',
  'event000001052-particles.csv',
  'event000001052-cells.csv',
  'event000001052-hits.csv'],
 '000001142': ['event000001142-cells.csv',
  'event000001142-particles.csv',
  'event000001142-truth.csv',
  'event000001142-hits.csv'],
 '000001215': ['event000001215-truth.csv',
  'event000001215-cells.csv',
  'event000001215-hits.csv',
  'event000001215-particles.csv'],
 '000001241': ['event000001241-hits.csv',
  'event000001241-particles.csv',
  'event000001241-cells.csv',
  'event000001241-truth.csv'],
 '000001395': ['event000001395-cells.csv',
  'event000001395-truth.csv',
  'event000001395-particles.csv',
  'event000001395-hits.csv'],
 '000001472': ['event000001472-particles.csv',
  'event000001472-cells.csv',
  'event000001472-truth.csv',
  'event000001472-hits.csv'],
 '000001540': ['event000001540-particles.

In [40]:
event_grouped_dataset_filenames_1 = read_dataset_filenames_offset(['datasets/unarch/train_1/'], 20)

In [41]:
event_grouped_dataset_filenames_1

{'000001015': ['event000001015-truth.csv',
  'event000001015-particles.csv',
  'event000001015-cells.csv',
  'event000001015-hits.csv'],
 '000001127': ['event000001127-particles.csv',
  'event000001127-cells.csv',
  'event000001127-hits.csv',
  'event000001127-truth.csv'],
 '000001191': ['event000001191-cells.csv',
  'event000001191-hits.csv',
  'event000001191-particles.csv',
  'event000001191-truth.csv'],
 '000001287': ['event000001287-truth.csv',
  'event000001287-hits.csv',
  'event000001287-particles.csv',
  'event000001287-cells.csv'],
 '000001343': ['event000001343-hits.csv',
  'event000001343-cells.csv',
  'event000001343-truth.csv',
  'event000001343-particles.csv'],
 '000001384': ['event000001384-truth.csv',
  'event000001384-particles.csv',
  'event000001384-cells.csv',
  'event000001384-hits.csv'],
 '000001399': ['event000001399-truth.csv',
  'event000001399-particles.csv',
  'event000001399-cells.csv',
  'event000001399-hits.csv'],
 '000001404': ['event000001404-hits.csv',

In [14]:
def create_event_df(event_id, particles_df, truth_df, cells_df, hits_df):
    #return pd.merge(truth_df, hits_df, on='hit_id')
    truth_hits_df = pd.merge(truth_df, hits_df, on='hit_id')
    particles_truth_hits_df = pd.merge(particles_df, truth_hits_df, on='particle_id', how='right')
    #return particles_truth_hits_df
    return pd.merge(cells_df, particles_truth_hits_df, on='hit_id', how='outer')

In [15]:
all(hits_df['hit_id'] == truth_df['hit_id'])

True

In [16]:
event_df = create_event_df(0, particles_df, truth_df, cells_df, hits_df)

In [17]:
event_df.shape

(664996, 26)